In [1]:
import bokeh
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from datetime import datetime
import seaborn as sns
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import gridplot,layout
from bokeh.models import FactorRange
import pandas as pd 
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from bokeh.models import Range1d

output_notebook()

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = None
pd.options.display.max_columns = None

Loading BokehJS ...

In [2]:
import bokeh
bokeh.__version__

'2.0.1'

In [3]:
description = pd.read_csv('Covid19Canada-master/codebook.csv')

n_cases = pd.read_csv('Covid19Canada-master/cases.csv')
n_death = pd.read_csv('Covid19Canada-master/mortality.csv')

n_recovered = pd.read_csv('Covid19Canada-master/recovered_cumulative.csv')
n_testing = pd.read_csv('Covid19Canada-master/testing_cumulative.csv')

color = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)] 

In [4]:
display(description)

,Variable,Description,Label
0,case_id,National Case ID Number,NaN
1,provincial_case_id,Provincial Case ID Number,NaN
2,age,"Age, if specific age not given then range prov...",NaN
3,sex,Sex,NaN
4,health_region,"Health region, if reported",NaN
5,province,Province,NaN
6,country,Country,NaN
7,date_report,Reported date (i.e. public announcement date) ...,NaN
8,report_week,Week of Report (Sundays are 1st day of week),NaN
9,travel_yn,Travel history (yes/no),"0=no, 1=yes, Not Reported"


# Number of Cases in Canada

In [31]:
display(n_cases.head(1))
n_cases.shape

,case_id,provincial_case_id,age,sex,health_region,province,country,date_report,report_week,travel_yn,travel_history_country,people
0,1,1,50-59,Male,Toronto,Ontario,Canada,2020-01-25,2020-01-19,1,China,1


(6320, 12)

In [7]:
n_cases = n_cases.iloc[:,:11]
n_cases['date_report'] = pd.to_datetime(n_cases['date_report'], dayfirst = True)
n_cases['report_week'] = pd.to_datetime(n_cases['report_week'], dayfirst = True)
n_cases['people'] = 1

,case_id,provincial_case_id,age,sex,health_region,province,country,date_report,report_week,travel_yn,travel_history_country
0,1,1,50-59,Male,Toronto,Ontario,Canada,2020-01-25,2020-01-19,1,China


In [9]:
all_gender = n_cases.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
all_gender_factors = pd.unique(all_gender['sex'])
all_gender_data = ColumnDataSource(all_gender)

all_age = n_cases.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
all_age_factors = pd.unique(all_age['age'])
all_age_data = ColumnDataSource(all_age)

sex_case = n_cases.loc[n_cases['sex'] != 'Not Reported']
sex_case = sex_case.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
sex_case_factors = pd.unique(sex_case['sex'])
sex_case_data = ColumnDataSource(sex_case)

age_case = n_cases.loc[n_cases['age'] != 'Not Reported']
age_case = age_case.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
age_case_factors = pd.unique(age_case['age'])
age_case_data = ColumnDataSource(age_case)

sex_age = n_cases.loc[(n_cases['age'] != 'Not Reported') & (n_cases['sex'] != 'Not Reported')]
sex_age = sex_age.groupby(['age', 'sex'])['people'].sum().reset_index()
sex_age['label'] = tuple(zip(sex_age['age'], sex_age['sex']))
sex_age_data = ColumnDataSource(sex_age)

In [10]:
plot1 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = all_gender_factors),background_fill_color="#fafafa")
plot2 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = all_age_factors),background_fill_color="#fafafa")
plot3 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = sex_case_factors),background_fill_color="#fafafa")
plot4 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = age_case_factors),background_fill_color="#fafafa")
plot5 = figure(plot_width = 990, plot_height = 300, x_range = FactorRange(factors = sex_age['label']),background_fill_color="#fafafa")


plot1.vbar(x = 'sex' , top = 'people' , width = 0.8, source = all_gender_data)
plot2.vbar(x = 'age' , top = 'people' , width = 0.8, source = all_age_data)
plot3.vbar(x = 'sex' , top = 'people' , width = 0.8, source = sex_case_data)
plot4.vbar(x = 'age' , top = 'people' , width = 0.8, source = age_case_data)
plot5.vbar(x = 'label' , top = 'people' , width = 0.8, source = sex_age_data,\
          fill_color=factor_cmap('label', factors= sex_age['sex'] , start=1, end=2, palette = Spectral6))

plot1.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot3.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot4.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot5.add_tools(HoverTool(tooltips = [('Sales', '@people')]))

plot2.xaxis.major_label_orientation = 1
plot4.xaxis.major_label_orientation = 1
plot5.xaxis.major_label_orientation = 1

grid = layout([[plot1,plot2],[plot3, plot4],[plot5]]) 


show(grid)

In [11]:
data_report = pd.DataFrame(n_cases['date_report'].value_counts()).reset_index().sort_values(by = 'index')
data_report_data = ColumnDataSource(data_report)
report_week = pd.DataFrame(n_cases['report_week'].value_counts()).reset_index().sort_values(by = 'index')
report_week_data = ColumnDataSource(report_week)

In [12]:
plot1 = figure(plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot2 = figure(plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot3 = figure(plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")

plot1.line(x = 'index' , y = 'date_report' , width = 2, source = data_report_data);
plot1.circle(x = 'index', y = 'date_report', size= 5, source = data_report_data)

plot2.line(x = 'index' , y = 'report_week' , width = 2, source = report_week_data);
plot2.circle(x = 'index', y = 'report_week', size= 5, source = report_week_data)

line1 = plot3.line(x = 'index' , y = 'date_report' , width = 2, source = data_report_data);
line2 = plot3.line(x = 'index' , y = 'report_week' , width = 2, source = report_week_data, color= 'orange');
circle1 = plot3.circle(x = 'index', y = 'date_report', size= 5, source = data_report_data)
circle2 = plot3.circle(x = 'index', y = 'report_week', size= 5, source = report_week_data,color= 'orange')


plot1.add_tools(HoverTool(tooltips = [('Sales', '@date_report')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@report_week')]))
plot3.add_tools(HoverTool(renderers=[line1], tooltips = [('Sales', '@date_report')]))
plot3.add_tools(HoverTool(renderers=[line2], tooltips = [('Sales', '@report_week')]))

grid = layout([[plot1],[plot2],[plot3]]) 

show(grid)

In [13]:
province = (pd.DataFrame(n_cases['province'].value_counts()).reset_index()
                 .sort_values(by = 'province',ascending = False))
province_factors = pd.unique(province['index'])
province_data = ColumnDataSource(province)

health_region = (pd.DataFrame(n_cases['health_region'].value_counts()).reset_index()
                 .sort_values(by = 'health_region',ascending = False))
health_region_factors = pd.unique(health_region['index'])
health_region_data = ColumnDataSource(health_region)

In [14]:
plot1 = figure(plot_width = 900, plot_height = 300,x_range = FactorRange(factors = province_factors),background_fill_color="#fafafa")
plot2 = figure(plot_width = 990, plot_height = 300,x_range = FactorRange(factors = health_region_factors),background_fill_color="#fafafa")


plot1.vbar(x = 'index' , top = 'province' , width = 0.8, source = province_data);
plot2.vbar(x = 'index' , top = 'health_region' , width = 0.8, source = health_region_data);
plot1.add_tools(HoverTool(tooltips = [('Sales', '@province')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@health_region')]))


plot1.xaxis.major_label_orientation = 1
plot2.xaxis.major_label_orientation = 1
grid = layout([[plot1],[plot2]]) 

show(grid)

In [15]:
n_travel = n_cases.loc[n_cases['travel_yn'] != 'Not Reported']
n_travel = (pd.DataFrame(n_travel['travel_yn'].value_counts()).reset_index()
                 .sort_values(by = 'travel_yn',ascending = False))
n_travel_factors = pd.unique(n_travel['index'])
n_travel_data = ColumnDataSource(n_travel)

travel_history_country = (pd.DataFrame(n_cases['travel_history_country'].value_counts()).reset_index()
                 .sort_values(by = 'travel_history_country',ascending = False))
travel_history_country_factors = pd.unique(travel_history_country['index'])
travel_history_country_data = ColumnDataSource(travel_history_country)

In [16]:
plot1 = figure(plot_width = 200, plot_height = 500,x_range = FactorRange(factors = n_travel_factors),background_fill_color="#fafafa")
plot2 = figure(plot_width = 780, plot_height = 500,x_range = FactorRange(factors = travel_history_country_factors),background_fill_color="#fafafa")


plot1.vbar(x = 'index' , top = 'travel_yn' , width = 0.2, source = n_travel_data);
plot2.vbar(x = 'index' , top = 'travel_history_country' , width = 0.8, source = travel_history_country_data);
plot1.add_tools(HoverTool(tooltips = [('Sales', '@travel_yn')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@travel_history_country')]))


plot2.xaxis.major_label_orientation = 1
grid = layout([[plot1,plot2]]) 

show(grid)

# Number of Deaths in Canada

In [17]:
display(n_death.head(1))

,death_id,province_death_id,case_id,age,sex,health_region,province,country,date_death_report,death_source,additional_info,additional_source
0,1,1,60.0,80-89,Male,Vancouver Coastal,BC,Canada,08-03-2020,https://news.gov.bc.ca/releases/2020HLTH0068-0...,Lynn Valley Resident,NaN


In [38]:
n_death['date_death_report'] = pd.to_datetime(n_death['date_death_report'], dayfirst = True)
n_death['people'] = 1

In [39]:
death_gender = n_death.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
death_gender_factors = pd.unique(death_gender['sex'])
death_gender_data = ColumnDataSource(death_gender)

death_age = n_death.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
death_age_factors = pd.unique(death_age['age'])
death_age_data = ColumnDataSource(death_age)

death_x = n_death.groupby(['age', 'sex'])['people'].sum().reset_index()
death_x['label'] = tuple(zip(death_x['age'], death_x['sex']))
death_x_data = ColumnDataSource(death_x)




In [40]:
plot1 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = death_gender_factors),background_fill_color="#fafafa")
plot2 = figure(plot_width = 495, plot_height = 300, x_range = FactorRange(factors = death_age_factors),background_fill_color="#fafafa")
plot3 = figure(plot_width = 990, plot_height = 300, x_range = FactorRange(factors = death_x['label']),background_fill_color="#fafafa")

plot1.vbar(x = 'sex' , top = 'people' , width = 0.5, source = death_gender_data)
plot2.vbar(x = 'age' , top = 'people' , width = 0.8, source = death_age_data)
plot3.vbar(x = 'label' , top = 'people' , width = 0.8, source = sex_age_data,\
          fill_color=factor_cmap('label', factors= sex_age['sex'] , start=1, end=2, palette = Spectral6))

plot1.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@people')]))
plot3.add_tools(HoverTool(tooltips = [('Sales', '@people')]))

grid = layout([[plot1,plot2],[plot3]]) 


show(grid)

In [44]:
death_report = pd.DataFrame(n_death['date_death_report'].value_counts()).reset_index().sort_values(by = 'index')
death_report_data = ColumnDataSource(death_report)

death_report

,index,date_death_report
14,2020-03-08,1
13,2020-03-11,1
7,2020-03-16,3
10,2020-03-17,3
12,2020-03-18,1
8,2020-03-19,3
15,2020-03-20,1
3,2020-03-21,5
11,2020-03-22,3
6,2020-03-23,3


In [42]:
plot1 = figure(plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot1.line(x = 'index' , y = 'date_death_report' , width = 2, source = death_report_data);
plot1.circle(x = 'index', y = 'date_death_report', size= 5, source = death_report_data)
plot1.add_tools(HoverTool(tooltips = [('Sales', '@date_death_report')]))

show(plot1)

In [32]:
province = (pd.DataFrame(n_death['province'].value_counts()).reset_index()
                 .sort_values(by = 'province',ascending = False))
province_factors = pd.unique(province['index'])
province_data = ColumnDataSource(province)

health_region = (pd.DataFrame(n_death['health_region'].value_counts()).reset_index()
                 .sort_values(by = 'health_region',ascending = False))
health_region_factors = pd.unique(health_region['index'])
health_region_data = ColumnDataSource(health_region)

In [33]:
plot1 = figure(plot_width = 900, plot_height = 300,x_range = FactorRange(factors = province_factors),background_fill_color="#fafafa")
plot2 = figure(plot_width = 990, plot_height = 300,x_range = FactorRange(factors = health_region_factors),background_fill_color="#fafafa")


plot1.vbar(x = 'index' , top = 'province' , width = 0.8, source = province_data);
plot2.vbar(x = 'index' , top = 'health_region' , width = 0.8, source = health_region_data);
plot1.add_tools(HoverTool(tooltips = [('Sales', '@province')]))
plot2.add_tools(HoverTool(tooltips = [('Sales', '@health_region')]))


plot1.xaxis.major_label_orientation = 1
plot2.xaxis.major_label_orientation = 1
grid = layout([[plot1],[plot2]]) 

show(grid)

# Number of Testing Case

In [21]:
n_testing['cumulative_testing'] = n_testing['cumulative_testing'].str.rstrip('*')
n_testing['cumulative_testing'] = n_testing['cumulative_testing'].fillna(0)

In [22]:
n_testing['date_testing'] = pd.to_datetime(n_testing['date_testing'], dayfirst = True)

In [23]:
n_testing = n_testing.sort_values(by = ['province','date_testing']);
n_testing['province'].unique()

array(['Alberta', 'BC', 'Manitoba', 'NL', 'NWT', 'New Brunswick',
       'Nova Scotia', 'Nunavut', 'Ontario', 'PEI', 'Quebec',
       'Saskatchewan', 'Yukon'], dtype=object)

In [24]:
test = {}
test_data = {}
for name in n_testing['province'].unique():
    test[name] = pd.DataFrame(n_testing.loc[n_testing['province'] == name])
    test_data[name] = ColumnDataSource(test[name])

In [25]:
plot3 = figure(plot_width = 900, plot_height = 900,x_axis_type="datetime",y_range=(0, 60000),background_fill_color="#fafafa")

line1 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Alberta'],color= color[0]);
line2 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['BC'], color= color[1]);
line3 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Manitoba'], color= color[2]);
line4 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NL'], color= color[3]);
line5 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NWT'], color= color[4]);
line6 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['New Brunswick'], color= color[5]);
line7 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[6]);
line8 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nunavut'], color= color[7]);
line9 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Ontario'], color= color[8]);
line10 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['PEI'], color= color[9]);
line11 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Quebec'], color= color[10]);
line12 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Saskatchewan'], color= color[11]);
line13 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[12]);
line14 = plot3.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Yukon'], color= color[13]);


circle1 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Alberta'],color = color[0]);
circle2 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['BC'], color= color[1]);
circle3 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Manitoba'], color= color[2]);
circle4 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NL'], color= color[3]);
circle5 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NWT'], color= color[4]);
circle6 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['New Brunswick'], color= color[5]);
circle7 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[6]);
circle8 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nunavut'], color= color[7]);
circle9 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Ontario'], color= color[8]);
circle10 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['PEI'], color= color[9]);
circle11 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Quebec'], color= color[10]);
circle12 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Saskatchewan'], color= color[11]);
circle13 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[12]);
circle14 = plot3.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Yukon'], color= color[13]);


line = [line1,line2,line3,line4,line5,line6,line7,line8,line9,line10,line11,line12,line13,line14]

for i in line:
    plot3.add_tools(HoverTool(renderers=[i], tooltips = [('Sales', '@cumulative_testing')]))


grid = layout([[plot3]]) 

show(grid)

# Number of Recoverd Case

In [26]:
display(n_recovered.head(1))

,date_recovered,province,cumulative_recovered
0,29-03-2020,Alberta,73.0


In [27]:
n_recovered['cumulative_recovered'] = n_recovered['cumulative_recovered'].fillna(0)
n_recovered['date_recovered'] = pd.to_datetime(n_recovered['date_recovered'], dayfirst = True)

In [28]:
n_recovered = n_recovered.sort_values(by = ['province','date_recovered']);
n_recovered['province'].unique()

array(['Alberta', 'BC', 'Manitoba', 'NL', 'NWT', 'New Brunswick',
       'Nova Scotia', 'Nunavut', 'Ontario', 'PEI', 'Quebec',
       'Saskatchewan', 'Yukon'], dtype=object)

In [29]:
recovered = {}
recovered_data = {}
for name in n_recovered['province'].unique():
    recovered[name] = pd.DataFrame(n_recovered.loc[n_recovered['province'] == name])
    recovered_data[name] = ColumnDataSource(recovered[name])

In [30]:
plot3 = figure(plot_width = 900, plot_height = 200,x_axis_type="datetime",y_range=(1, 420),background_fill_color="#fafafa")

line1 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Alberta'],color = color[0]);
line2 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['BC'], color= color[1]);
line3 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Manitoba'], color= color[2]);
line4 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NL'], color= color[3]);
line5 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NWT'], color= color[4]);
line6 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['New Brunswick'], color= color[5]);
line7 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[6]);
line8 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nunavut'], color= color[7]);
line9 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Ontario'], color= color[8]);
line10 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['PEI'], color= color[9]);
line11 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Quebec'], color= color[10]);
line12 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Saskatchewan'], color= color[11]);
line13 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[12]);
line14 = plot3.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Yukon'], color= color[13]);



circle1 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Alberta'],color = color[0]);
circle2 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['BC'], color= color[1]);
circle3 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Manitoba'], color= color[2]);
circle4 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NL'], color= color[3]);
circle5 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NWT'], color= color[4]);
circle6 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['New Brunswick'], color= color[5]);
circle7 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[6]);
circle8 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nunavut'], color= color[7]);
circle9 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Ontario'], color= color[8]);
circle10 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['PEI'], color= color[9]);
circle11 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Quebec'], color= color[10]);
circle12 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Saskatchewan'], color= color[11]);
circle13 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[12]);
circle14 = plot3.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Yukon'], color= color[13]);



line = [line1,line2,line3,line4,line5,line6,line7,line8,line9,line10,line11,line12,line13,line14]

for i in line:
    plot3.add_tools(HoverTool(renderers=[i], tooltips = [('Sales', '@cumulative_recovered')]))



grid = layout([[plot3]]) 

show(grid)